In [19]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance
import re
from typing import List, Dict


# Load environment variables
load_dotenv()

QDRANT_HOST = os.getenv("QDRANT_HOST")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_COLLECTION = os.getenv("QDRANT_COLLECTION", "srt_subtitles")



In [20]:
# Function to parse .srt file
def parse_srt(file_path: str, movie_name: str, idx:int=0) -> List[Dict[str, str]]:
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    pattern = re.compile(r'(\d+)\s+(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\s+(.+?)(?=\n\d+\n|\Z)', re.DOTALL)
    matches = pattern.findall(content)
    subtitles = []
    for _, start, end, text in matches:
        subtitles.append({
            "index": int(idx),
            "start": start,
            "end": end,
            "movie_name": movie_name,
            "text": text.replace('\n', ' ').strip()
        })
        idx += 1
    return subtitles


SRT_PATH = "/Users/enkhbat_1/projects/ai-video-ge/movie-reels/srt_files/"
raw_srt_file = parse_srt(f"{SRT_PATH}Shawshank_Redemption_1.srt",movie_name="Shawshank Redemption")
print(raw_srt_file[1:10])  

[{'index': 1, 'start': '00:02:08,080', 'end': '00:02:12,240', 'movie_name': 'Shawshank Redemption', 'text': '...the confrontation you had with your wife the night she was murdered.'}, {'index': 2, 'start': '00:02:16,720', 'end': '00:02:18,320', 'movie_name': 'Shawshank Redemption', 'text': 'It was very bitter.'}, {'index': 3, 'start': '00:02:18,640', 'end': '00:02:22,760', 'movie_name': 'Shawshank Redemption', 'text': 'She said she was glad I knew, that she hated all the sneaking around.'}, {'index': 4, 'start': '00:02:24,280', 'end': '00:02:27,600', 'movie_name': 'Shawshank Redemption', 'text': 'And she said that she wanted a divorce in Reno.'}, {'index': 5, 'start': '00:02:27,840', 'end': '00:02:31,360', 'movie_name': 'Shawshank Redemption', 'text': '-What was your response? -I told her I would not grant one.'}, {'index': 6, 'start': '00:02:31,560', 'end': '00:02:34,600', 'movie_name': 'Shawshank Redemption', 'text': '"I\'ll see you in hell before I see you in Reno."'}, {'index': 7, 

In [21]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

def prepare_documents(raw_srt_file: List[dict]) -> List[Document]:
    """
    Convert raw SRT file data to LangChain Document objects.
    
    Args:
        raw_srt_file (list): List of dictionaries containing subtitle data.
        
    Returns:
        list: List of LangChain Document objects.
    """

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100, chunk_overlap=10, add_start_index=True
    )
    return text_splitter.split_documents([
        Document(
            page_content=sub["text"],
            metadata={
                "index": sub["index"],
                "start": sub["start"],
                "end": sub["end"],
                "movie_name": sub["movie_name"]
            }
        )
        for sub in raw_srt_file
    ])

# Convert raw_srt_file to LangChain Document objects
all_splits = prepare_documents(raw_srt_file)

len(all_splits)  # Total number of splits

930

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings



embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B",
                                   cache_folder= "/Users/enkhbat_1/projects/ai-video-ge/cache-models/")


# Example: simple embedding (replace with your embedding model)

def embed_text(text):
    # Dummy embedding: length of text as vector
    return [float(len(text))]

In [24]:
len(embeddings.embed_query(all_splits[0].page_content))  # Example embedding for the first split

1024

In [20]:
client = QdrantClient(
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY
)


In [22]:
# Read and store SRT subtitles
def store_srt_to_qdrant(srt_path):
    subtitles = parse_srt(srt_path)
    points = []
    for sub in subtitles:
        vector = embed_text(sub["text"])
        payload = {
            "index": sub["index"],
            "start": sub["start"],
            "end": sub["end"],
            "text": sub["text"]
        }
        points.append(PointStruct(id=sub["index"], vector=vector, payload=payload))
    client.upsert(collection_name=QDRANT_COLLECTION, points=points)

In [ ]:
### Chat with Gemini model
import os

from langchain.chat_models import init_chat_model

# Use Gemini instead of Gemma for system prompt support
model = init_chat_model(model="gemini-1.5-flash",
                         model_provider="google_genai", 
                         api_key=os.environ["GEMINI_API_KEY"])


from langchain_core.messages import HumanMessage

# For Gemma models, combine system instruction with human message
messages = [
    HumanMessage("Translate the following from English into Italian: hi!"),
]

model.invoke(messages).content
